In [1]:
!pip install --upgrade pandas

     |████████████████████████████████| 10.0MB 3.5MB/s eta 0:00:01
  Found existing installation: pandas 0.19.2
    Uninstalling pandas-0.19.2:
      Successfully uninstalled pandas-0.19.2
You should consider upgrading via the 'pip install --upgrade pip' command.


In [2]:
!pip install pyarrow

You should consider upgrading via the 'pip install --upgrade pip' command.


In [3]:
import pandas as pd
import numpy as np
from numpy import nan as NA

from io import StringIO
from azure.storage.blob import BlockBlobService

In [4]:
import pyarrow.parquet as pq
from io import BytesIO

In [156]:
YearMonth = '2018-06-01'

In [157]:
#Azure account credentials
azure_storage_account_name = "agedatasciencecocstorage"
azure_storage_account_key = "V/FXG+lIWtYc854wbRA8BxGCLGJijS0NDwLvNHfmAxqRopAL3WHWGIQoHxpArG9lZoX+iY+dvtPYoB379w4MIw=="

if azure_storage_account_name is None or azure_storage_account_key is None:
    raise Exception("You must provide a name and key for an Azure Storage account")

In [158]:
# First, we need to connect to our blob via the BlobService
blob_service = BlockBlobService(azure_storage_account_name, azure_storage_account_key)
# There are also methods to list containers and blobs
#containers = blob_service.list_containers()
blobs = blob_service.list_blobs('banking-dm')

In [159]:
# List the blobs in the container.
print("\nList blobs in the container")
#generator = block_blob_service.list_blobs(container_name)
for blob in blobs:
    print("\t Blob name: " + blob.name)


List blobs in the container
	 Blob name: ailiev/Transactions_Master_2018-01-01.parquet.gzip
	 Blob name: ailiev/Transactions_Master_2018-02-01.parquet.gzip
	 Blob name: ailiev/Transactions_Master_2018-03-01.parquet.gzip
	 Blob name: ailiev/Transactions_Master_2018-04-01.parquet.gzip
	 Blob name: ailiev/Transactions_Master_2018-05-01.parquet.gzip
	 Blob name: ailiev/sample.txt
	 Blob name: data/Customer_economic_sector_lookup.csv
	 Blob name: data/Customer_information_2018-01-01.parquet.gzip
	 Blob name: data/Customer_information_2018-02-01.parquet.gzip
	 Blob name: data/Customer_information_2018-03-01.parquet.gzip
	 Blob name: data/Customer_information_2018-04-01.parquet.gzip
	 Blob name: data/Customer_information_2018-05-01.parquet.gzip
	 Blob name: data/Customer_information_2018-06-01.parquet.gzip
	 Blob name: data/Monthly_Balance_2018-01-01.parquet.gzip
	 Blob name: data/Monthly_Balance_2018-02-01.parquet.gzip
	 Blob name: data/Monthly_Balance_2018-03-01.parquet.gzip
	 Blob name: d

In [160]:
# Insider a container we can create other containers or a blob
blob_service.create_blob_from_text('banking-dm', 'ailiev/sample.txt', 'Develop Data Mart by using the data folder files as input.')

In [161]:
# We can also read our blob from azure and get the text.
blob_service.get_blob_to_path('banking-dm', 'ailiev/sample.txt', 'sample.txt')

!cat sample.txt

Develop Data Mart by using the data folder files as input.

### Read Data

In [162]:
container_name = "banking-dm"
blob_name_parquet = 'data/Transactions_' + YearMonth + '.parquet.gzip'
blob_name = 'data/transaction_codes_lookup.csv'

In [163]:
byte_stream = BytesIO()
try:
   blob_service.get_blob_to_stream(container_name=container_name, blob_name=blob_name_parquet, stream=byte_stream)
   transactions = pq.read_table(source=byte_stream).to_pandas()
   # Do work on df ...
finally:
   # Add finally block to ensure closure of the stream
   byte_stream.close()

In [164]:
#path where files are read in from
#input_path = 'C:/Users/Aleksandar.Iliev/Desktop/Projects/Data Science CoC/Banking_Data_Mart/Data/25022019/'

In [165]:
#path where output is written in 
#output_path = input_path + 'xls/'

In [166]:
#read in csv file
#transactions = pd.read_csv(str(input_path + 'Transactions.csv'), header=0, low_memory=False)

In [167]:
transactions.shape

(2481352, 8)

In [168]:
blobstring = blob_service.get_blob_to_text(container_name, blob_name).content
codes = pd.read_csv(StringIO(blobstring), header=0, sep=';')

In [169]:
#codes = pd.read_csv(str(input_path + 'transaction_codes_lookup.csv'), header=0, sep=';')

In [170]:
codes['Card'] = np.where(codes['Code'].isin(["TRA05", 'TRA11']), "DC", 
                        np.where(codes['Code'].isin(["TRA06", 'TRA12']), "CC", "LD"))

In [171]:
codes

,Channel,Transaction Type,Flag,Code,Card
0,Branch,Deposits,Credit,TRA01,LD
1,Branch,Cash Withdrawals,Debit,TRA02,LD
2,Branch,Transfers,Debit,TRA03,LD
3,ATM,Deposits,Credit,TRA04,LD
4,ATM,Cash Withdrawals,Debit,TRA05,DC
5,ATM,Cash Withdrawals,Debit,TRA06,CC
6,Internet,Transfers,Debit,TRA07,LD
7,Other,Deposits,Credit,TRA08,LD
8,Other,Cash Withdrawals,Debit,TRA09,LD
9,Other,Transfers,Debit,TRA10,LD


In [172]:
trans_codes = transactions.merge(codes, how='left', left_on='Transaction_code', right_on='Code')

In [173]:
trans_codes.shape

(2481352, 13)

In [174]:
trans_codes.head()

,YearMonth,Customer_ID,Product_code,Account_ID,Transaction_date,Transaction_code,Transaction_amount,Bordero_number,Channel,Transaction Type,Flag,Code,Card
0,2018-06-01,47178,PRO09,100011,2018-05-05,TRA08,619.16,47178100011PRO0912018-05-05,Other,Deposits,Credit,TRA08,LD
1,2018-06-01,77103,PRO10,100029,2018-05-04,TRA01,233.96,77103100029PRO1022018-05-04,Branch,Deposits,Credit,TRA01,LD
2,2018-06-01,77868,PRO09,100079,2018-01-02,TRA01,203.99,77868100079PRO0932018-01-02,Branch,Deposits,Credit,TRA01,LD
3,2018-06-01,90122,PRO09,100085,2018-05-04,TRA08,257.99,90122100085PRO0942018-05-04,Other,Deposits,Credit,TRA08,LD
4,2018-06-01,18818,PRO09,100094,2018-06-07,TRA01,491.68,18818100094PRO0952018-06-07,Branch,Deposits,Credit,TRA01,LD


In [175]:
pd.value_counts(trans_codes.Channel)

ATM         931336
POS         890115
Other       277548
Branch      273732
Internet    108621
Name: Channel, dtype: int64

Aggregations

In [176]:
def aggregated_index(df, dic):
    arr = []
    for val in df.columns.values:
        arr.append([dic.get(n, n) for n in val])
    columns = pd.Index("_".join(i) for i in arr)
    columns = [x.upper() for x in columns]
    return columns

In [177]:
aggreg_channel = pd.pivot_table(trans_codes, values=['Transaction_amount'], index=['Customer_ID'], columns=['Card', 'Channel'], 
                             aggfunc={'Transaction_amount': [sum, len]})

In [178]:
dic_channel = {'Transaction_amount':'', 'len':'NUTN', 'sum':'TNAM'}

In [179]:
col_agg_channel = aggregated_index(aggreg_channel, dic_channel)
col_agg_channel = [item[1:] for item in col_agg_channel]
aggreg_channel.columns = col_agg_channel
aggreg_channel.reset_index(inplace=True)

In [180]:
aggreg_channel.head(10)

,Customer_ID,NUTN_CC_ATM,NUTN_CC_POS,NUTN_DC_ATM,NUTN_DC_POS,NUTN_LD_ATM,NUTN_LD_BRANCH,NUTN_LD_INTERNET,NUTN_LD_OTHER,TNAM_CC_ATM,TNAM_CC_POS,TNAM_DC_ATM,TNAM_DC_POS,TNAM_LD_ATM,TNAM_LD_BRANCH,TNAM_LD_INTERNET,TNAM_LD_OTHER
0,2,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.00,NaN,NaN,NaN,NaN
1,3,NaN,NaN,5.0,8.0,1.0,5.0,7.0,7.0,NaN,NaN,1986.40,1662.42,426.0,2261.47,9087.46,1927.70
2,4,24.0,18.0,NaN,NaN,2.0,NaN,NaN,NaN,2002.91,633.01,NaN,NaN,1068.0,NaN,NaN,NaN
3,5,15.0,11.0,NaN,NaN,NaN,NaN,NaN,2.0,707.46,469.47,NaN,NaN,NaN,NaN,NaN,316.00
4,6,24.0,17.0,NaN,NaN,1.0,2.0,NaN,5.0,4933.99,1270.44,NaN,NaN,238.0,761.00,NaN,2045.00
5,7,NaN,NaN,12.0,8.0,NaN,NaN,NaN,NaN,NaN,NaN,81944.57,3564.54,NaN,NaN,NaN,NaN
6,8,1.0,4.0,NaN,NaN,NaN,NaN,NaN,NaN,17.77,285.70,NaN,NaN,NaN,NaN,NaN,NaN
7,11,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,42.46
8,12,NaN,NaN,NaN,NaN,NaN,5.0,2.0,2.0,NaN,NaN,NaN,NaN,NaN,1690.00,100.00,140.00
9,14,40.0,47.0,NaN,NaN,1.0,NaN,NaN,NaN,2596.98,2516.29,NaN,NaN,366.0,NaN,NaN,NaN


In [181]:
aggreg_channel.shape

(75417, 17)

In [182]:
aggreg_type = pd.pivot_table(trans_codes, values=['Transaction_amount'], index=['Customer_ID'], columns=['Card', 'Channel', 'Flag'], 
                             aggfunc={'Transaction_amount': [sum, len]})

In [183]:
dic_type = {'Transaction_amount':'', 'len':'NUTN', 'sum':'TNAM', 'Debit':'DT', 'Credit':'CT'}

In [184]:
col_agg_type = aggregated_index(aggreg_type, dic_type)
col_agg_type = [item[1:] for item in col_agg_type]
aggreg_type.columns = col_agg_type
aggreg_type.reset_index(inplace=True)

In [185]:
aggreg_type.head()

,Customer_ID,NUTN_CC_ATM_DT,NUTN_CC_POS_DT,NUTN_DC_ATM_DT,NUTN_DC_POS_DT,NUTN_LD_ATM_CT,NUTN_LD_BRANCH_CT,NUTN_LD_BRANCH_DT,NUTN_LD_INTERNET_DT,NUTN_LD_OTHER_CT,...,TNAM_CC_ATM_DT,TNAM_CC_POS_DT,TNAM_DC_ATM_DT,TNAM_DC_POS_DT,TNAM_LD_ATM_CT,TNAM_LD_BRANCH_CT,TNAM_LD_BRANCH_DT,TNAM_LD_INTERNET_DT,TNAM_LD_OTHER_CT,TNAM_LD_OTHER_DT
0,2,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,0.00,NaN,NaN,NaN,NaN,NaN,NaN
1,3,NaN,NaN,5.0,8.0,1.0,NaN,5.0,7.0,1.0,...,NaN,NaN,1986.4,1662.42,426.0,NaN,2261.47,9087.46,426.0,1501.7
2,4,24.0,18.0,NaN,NaN,2.0,NaN,NaN,NaN,NaN,...,2002.91,633.01,NaN,NaN,1068.0,NaN,NaN,NaN,NaN,NaN
3,5,15.0,11.0,NaN,NaN,NaN,NaN,NaN,NaN,2.0,...,707.46,469.47,NaN,NaN,NaN,NaN,NaN,NaN,316.0,NaN
4,6,24.0,17.0,NaN,NaN,1.0,2.0,NaN,NaN,5.0,...,4933.99,1270.44,NaN,NaN,238.0,761.0,NaN,NaN,2045.0,NaN


In [186]:
ld_trans = trans_codes[trans_codes['Card']=='LD']

In [187]:
trans_codes.shape

(2481352, 13)

In [188]:
ld_trans.shape

(699573, 13)

In [189]:
#only Loan & Deposits transactions
aggreg_ld_type = pd.pivot_table(ld_trans, values=['Transaction_amount'], index=['Customer_ID'], columns=['Card', 'Transaction Type'], 
                             aggfunc={'Transaction_amount': [sum, len]})

In [190]:
dic_ld_type  = {'Transaction_amount':'', 'Transaction Type':'TYPE', 'len':'NUTN', 'sum':'TNAM', 'Debit':'DT', 'Credit':'CT',
      'Cash Withdrawals':'Withdrawals'}

In [191]:
col_ld_type = aggregated_index(aggreg_ld_type, dic_ld_type)
col_ld_type = [item[1:] for item in col_ld_type]
aggreg_ld_type.columns = col_ld_type
aggreg_ld_type.reset_index(inplace=True)

In [192]:
aggreg_ld_type.head()

,Customer_ID,NUTN_LD_WITHDRAWALS,NUTN_LD_DEPOSITS,NUTN_LD_TRANSFERS,TNAM_LD_WITHDRAWALS,TNAM_LD_DEPOSITS,TNAM_LD_TRANSFERS
0,3,5.0,2.0,13.0,1925.85,852.00,10924.78
1,4,NaN,2.0,NaN,NaN,1068.00,NaN
2,5,NaN,2.0,NaN,NaN,316.00,NaN
3,6,NaN,8.0,NaN,NaN,3044.00,NaN
4,11,NaN,1.0,NaN,NaN,42.46,NaN


In [193]:
aggreg_days = pd.pivot_table(trans_codes, values=['Transaction_date'], index=['Customer_ID'], columns=['Card'],
                             aggfunc={'Transaction_date': [lambda x: len(x.unique())]})

In [194]:
dic_days = {'Transaction_date':'DIFF_DAYS', 'Transaction Type':'TYPE', '<lambda>':'TR', 'sum':'TNAM', 'Debit':'DT', 'Credit':'CT',
      'Cash Withdrawals':'Withdrawals'}

In [195]:
col_days = aggregated_index(aggreg_days, dic_days)
#col_days = [item[1:] for item in col_ld_type]
aggreg_days.columns = col_days
aggreg_days.reset_index(inplace=True)

In [196]:
aggreg_days.head()

,Customer_ID,DIFF_DAYS_TR_CC,DIFF_DAYS_TR_DC,DIFF_DAYS_TR_LD
0,2,NaN,1.0,NaN
1,3,NaN,10.0,14.0
2,4,22.0,NaN,2.0
3,5,17.0,NaN,2.0
4,6,21.0,NaN,7.0


In [197]:
aggreg_days_any = pd.pivot_table(trans_codes, values=['Transaction_date'], index=['Customer_ID'],
                             aggfunc={'Transaction_date': [lambda x: len(x.unique())]})

In [198]:
col_days_any = aggregated_index(aggreg_days_any, dic_days)
#col_days = [item[1:] for item in col_ld_type]
aggreg_days_any.columns = col_days_any
aggreg_days_any.reset_index(inplace=True)

In [199]:
aggreg_days_any.head()

,Customer_ID,DIFF_DAYS_TR
0,2,1
1,3,20
2,4,23
3,5,18
4,6,22


In [200]:
aggreg_dt_ct = pd.pivot_table(trans_codes, values=['Transaction_amount'], index=['Customer_ID'], columns=['Flag'], 
                             aggfunc={'Transaction_amount': [sum, len]})

In [201]:
dic_dt_ct = {'Transaction_amount':'', 'len':'NUTN', 'sum':'TNAM','Debit':'DT', 'Credit':'CT'}

In [202]:
col_agg_dt_ct = aggregated_index(aggreg_dt_ct, dic_dt_ct)
col_agg_dt_ct = [item[1:] for item in col_agg_dt_ct]
aggreg_dt_ct.columns = col_agg_dt_ct
aggreg_dt_ct.reset_index(inplace=True)

In [203]:
aggreg_dt_ct.head()

,Customer_ID,NUTN_CT,NUTN_DT,TNAM_CT,TNAM_DT
0,2,NaN,1.0,NaN,0.00
1,3,2.0,31.0,852.0,16499.45
2,4,2.0,42.0,1068.0,2635.92
3,5,2.0,26.0,316.0,1176.93
4,6,8.0,41.0,3044.0,6204.43


In [204]:
aggreg_card = pd.pivot_table(trans_codes, values=['Transaction_amount'], index=['Customer_ID'], columns=['Card'], 
                             aggfunc={'Transaction_amount': [sum, len]})

In [205]:
dic_card = {'Transaction_amount':'', 'len':'NUTN', 'sum':'TNAM'}

In [206]:
col_agg_card = aggregated_index(aggreg_card, dic_dt_ct)
col_agg_card = [item[1:] for item in col_agg_card]
aggreg_card.columns = col_agg_card
aggreg_card.reset_index(inplace=True)

In [207]:
aggreg_card.head()

,Customer_ID,NUTN_CC,NUTN_DC,NUTN_LD,TNAM_CC,TNAM_DC,TNAM_LD
0,2,NaN,1.0,NaN,NaN,0.00,NaN
1,3,NaN,13.0,20.0,NaN,3648.82,13702.63
2,4,42.0,NaN,2.0,2635.92,NaN,1068.00
3,5,26.0,NaN,2.0,1176.93,NaN,316.00
4,6,41.0,NaN,8.0,6204.43,NaN,3044.00


In [208]:
aggreg_total = pd.pivot_table(trans_codes, values=['Transaction_amount'], index=['Customer_ID'], 
                             aggfunc={'Transaction_amount': [sum, len]})

In [209]:
dic_total = {'Transaction_amount':'TOTAL', 'len':'NUTN', 'sum':'TNAM'}

In [210]:
col_total = aggregated_index(aggreg_total, dic_total)
aggreg_total.columns = col_total
aggreg_total.reset_index(inplace=True)

In [211]:
aggreg_total.head()

,Customer_ID,TOTAL_NUTN,TOTAL_TNAM
0,2,1.0,0.00
1,3,33.0,17351.45
2,4,44.0,3703.92
3,5,28.0,1492.93
4,6,49.0,9248.43


In [212]:
trans_agg_all = aggreg_channel.merge(aggreg_type,how='outer', on='Customer_ID')
trans_agg_all.shape

(75417, 37)

In [213]:
trans_agg_all = trans_agg_all.merge(aggreg_ld_type,how='outer', on='Customer_ID')
trans_agg_all.shape

(75417, 43)

In [214]:
trans_agg_all = trans_agg_all.merge(aggreg_days_any,how='outer', on='Customer_ID')
trans_agg_all.shape

(75417, 44)

In [215]:
trans_agg_all = trans_agg_all.merge(aggreg_days,how='outer', on='Customer_ID')
trans_agg_all.shape

(75417, 47)

In [216]:
trans_agg_all = trans_agg_all.merge(aggreg_dt_ct,how='outer', on='Customer_ID')
trans_agg_all.shape

(75417, 51)

In [217]:
trans_agg_all = trans_agg_all.merge(aggreg_card,how='outer', on='Customer_ID')
trans_agg_all.shape

(75417, 57)

In [218]:
trans_agg_all = trans_agg_all.merge(aggreg_total,how='outer', on='Customer_ID')
trans_agg_all.shape

(75417, 59)

Additional KPIs

In [219]:
#PCT
trans_agg_all['PCT_NUTN_CT'] = trans_agg_all['NUTN_CT']/trans_agg_all['TOTAL_NUTN']
trans_agg_all['PCT_NUTN_DT'] = trans_agg_all['NUTN_DT']/trans_agg_all['TOTAL_NUTN']
trans_agg_all['PCT_TNAM_CT'] = trans_agg_all['TNAM_CT']/trans_agg_all['TOTAL_TNAM']
trans_agg_all['PCT_TNAM_DT'] = trans_agg_all['TNAM_DT']/trans_agg_all['TOTAL_TNAM']

trans_agg_all['PCT_NUTN_CC'] = trans_agg_all['NUTN_CC']/trans_agg_all['TOTAL_NUTN']
trans_agg_all['PCT_NUTN_DC'] = trans_agg_all['NUTN_DC']/trans_agg_all['TOTAL_NUTN']
trans_agg_all['PCT_NUTN_LD'] = trans_agg_all['NUTN_LD']/trans_agg_all['TOTAL_NUTN']

trans_agg_all['PCT_TNAM_CC'] = trans_agg_all['TNAM_CC']/trans_agg_all['TOTAL_TNAM']
trans_agg_all['PCT_TNAM_DC'] = trans_agg_all['TNAM_DC']/trans_agg_all['TOTAL_TNAM']
trans_agg_all['PCT_TNAM_LD'] = trans_agg_all['TNAM_LD']/trans_agg_all['TOTAL_TNAM']

In [220]:
trans_agg_all['PCT_DAYS_TR_CC'] = trans_agg_all['DIFF_DAYS_TR_CC']/trans_agg_all['DIFF_DAYS_TR']
trans_agg_all['PCT_DAYS_TR_DC'] = trans_agg_all['DIFF_DAYS_TR_DC']/trans_agg_all['DIFF_DAYS_TR']
trans_agg_all['PCT_DAYS_TR_LD'] = trans_agg_all['DIFF_DAYS_TR_LD']/trans_agg_all['DIFF_DAYS_TR']

In [221]:
trans_agg_all['YearMonth'] = YearMonth

In [222]:
trans_agg_all.head()

,Customer_ID,NUTN_CC_ATM,NUTN_CC_POS,NUTN_DC_ATM,NUTN_DC_POS,NUTN_LD_ATM,NUTN_LD_BRANCH,NUTN_LD_INTERNET,NUTN_LD_OTHER,TNAM_CC_ATM,...,PCT_NUTN_CC,PCT_NUTN_DC,PCT_NUTN_LD,PCT_TNAM_CC,PCT_TNAM_DC,PCT_TNAM_LD,PCT_DAYS_TR_CC,PCT_DAYS_TR_DC,PCT_DAYS_TR_LD,YearMonth
0,2,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,...,NaN,1.000000,NaN,NaN,NaN,NaN,NaN,1.0,NaN,2018-06-01
1,3,NaN,NaN,5.0,8.0,1.0,5.0,7.0,7.0,NaN,...,NaN,0.393939,0.606061,NaN,0.210289,0.789711,NaN,0.5,0.700000,2018-06-01
2,4,24.0,18.0,NaN,NaN,2.0,NaN,NaN,NaN,2002.91,...,0.954545,NaN,0.045455,0.711657,NaN,0.288343,0.956522,NaN,0.086957,2018-06-01
3,5,15.0,11.0,NaN,NaN,NaN,NaN,NaN,2.0,707.46,...,0.928571,NaN,0.071429,0.788336,NaN,0.211664,0.944444,NaN,0.111111,2018-06-01
4,6,24.0,17.0,NaN,NaN,1.0,2.0,NaN,5.0,4933.99,...,0.836735,NaN,0.163265,0.670863,NaN,0.329137,0.954545,NaN,0.318182,2018-06-01


In [223]:
#!pip install --upgrade pandas

In [224]:
def write_pandas_dataframe_to_blob(blob_service, df, container_name, blob_name):
    """ Write Pandas dataframe to blob storage """
    buffer = BytesIO()
    df.to_parquet(buffer)
    blob_service.create_blob_from_bytes(
        container_name=container_name, blob_name=blob_name, blob=buffer.getvalue()
    )

In [225]:
write_to_blob_parquet = 'ailiev/Transactions_Master_' + YearMonth + '.parquet.gzip' #Transactions_Master_2018-01-01.parquet.gzip

In [226]:
# Write to blob using pyarrow
write_pandas_dataframe_to_blob(blob_service, trans_agg_all, 'banking-dm', write_to_blob_parquet)

In [227]:
#!pip list#

In [228]:
#check#

In [229]:
#read in csv file
#byte_stream = BytesIO()
#try:
#   blob_service.get_blob_to_stream(container_name=container_name, blob_name='ailiev/Transactions_Master.parquet.gzip', stream=byte_stream)
#   check = pq.read_table(source=byte_stream).to_pandas()
   # Do work on df ...
#finally:
   # Add finally block to ensure closure of the stream
#   byte_stream.close()

In [230]:
#check.head()